In [1]:
import pandas as pd
import requests
from zipfile import ZipFile
from io import BytesIO
import os

In [3]:
os.makedirs('./enem_2020', exist_ok=True)

In [7]:
url = 'https://download.inep.gov.br/microdados/microdados_enem_2020.zip'

filebytes = BytesIO(requests.get(url, verify=False).content)

myzip =  ZipFile(filebytes)
myzip.extractall('./enem_2020')

/home/phllp/dev/bootcamp_data_eng/env/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
path = '/home/phllp/dev/bootcamp_data_eng/modulo_04/dasafio_inep/enem_2020/DADOS/MICRODADOS_ENEM_2020.csv'
df = pd.read_csv(
  filepath_or_buffer=path, 
  delimiter=';', 
  decimal=',',
  encoding_errors='ignore',
  nrows=100000)
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,200006271946,2020,11,F,1,2,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001195856,2020,11,M,2,3,1,1,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001943954,2020,4,F,2,3,2,2,0,2,...,B,A,A,B,A,A,A,A,A,A
3,200001908998,2020,2,M,1,3,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001634757,2020,4,F,1,3,2,1,1,1,...,A,A,A,B,A,B,B,A,A,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,200006422434,2020,2,M,1,3,1,2,0,2,...,A,A,A,B,B,A,C,A,A,A
99996,200002731648,2020,11,F,1,2,1,1,0,1,...,B,A,A,B,A,A,D,A,A,B
99997,200006641817,2020,3,M,1,1,1,1,1,1,...,B,A,A,B,A,B,E,B,B,B
99998,200005653273,2020,5,M,1,3,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
path = '/home/phllp/dev/bootcamp_data_eng/modulo_04/dasafio_inep/enem_2020/DADOS/MICRODADOS_ENEM_2020.csv'
cols = [
  'TP_SEXO',
  'CO_UF_ESC',
  'CO_MUNICIPIO_ESC',
  'NU_NOTA_MT',
  'TP_ESCOLA'
]

df = pd.read_csv(
  filepath_or_buffer=path, 
  delimiter=';', 
  decimal=',',
  encoding_errors='ignore',
  usecols=cols
  )
df


,TP_SEXO,TP_ESCOLA,CO_MUNICIPIO_ESC,CO_UF_ESC,NU_NOTA_MT
0,F,1,NaN,NaN,NaN
1,M,1,NaN,NaN,711.3
2,F,2,2927408.0,29.0,NaN
3,M,2,3547304.0,35.0,759.4
4,F,1,NaN,NaN,NaN
...,...,...,...,...,...
5783104,F,2,2304103.0,23.0,725.7
5783105,M,2,5008305.0,50.0,719.5
5783106,F,1,NaN,NaN,648.1
5783107,F,3,5103403.0,51.0,NaN


# Filtrar apenas os alunos que estudaram em escolas no estado de Minas Gerais

- CO_UF_ESC: Código da Unidade da Federação da escola
- Código IBGE Minas Gerais = 31

In [6]:
df_mg = df.loc[df['CO_UF_ESC'] == 31.0]

### 1) Qual é a média da nota em matemática dos alunos que estudaram numa escola em Minas Gerais?

- NU_NOTA_MT: Nota da prova de Matemática

In [22]:
media = pd.to_numeric(df_mg['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 579.6812230047448


### 2) Qual é a média da nota em Linguagens e Códigos dos alunos que estudaram numa escola em Minas Gerais?

- NU_NOTA_LC: Nota da prova de Linguagens e Códigos

In [23]:
media = pd.to_numeric(df_mg['NU_NOTA_LC']).mean()
print(f'Media = {media}')

Media = 553.0102913021102


### 3) Qual é a média da nota em Ciências Humanas dos alunos do sexo FEMININO que estudaram numa escola em Minas Gerais?

- NU_NOTA_CH: Nota da prova de Ciências Humanas

In [27]:
df_fem_mg = df_mg.loc[df_mg['TP_SEXO'] == 'F']
media = pd.to_numeric(df_fem_mg['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 541.3084611723356


### 4) Qual é a média da nota em Ciências Humanas dos alunos do sexo MASCULINO que estudaram numa escola em Minas Gerais?

In [28]:
df_fem_mg = df_mg.loc[df_mg['TP_SEXO'] == 'M']
media = pd.to_numeric(df_fem_mg['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 559.5546844135076


### 5) Qual é a média da nota em Matemática dos alunos do sexo FEMININO que estudaram numa escola na cidade de Montes Claros?


- Codigo de Montes Claros no IBGE: 3143302

In [38]:
df_montes_claros = df_mg.loc[df_mg['CO_MUNICIPIO_ESC'] == 3143302]
df_montes_claros_fem = df_montes_claros.loc[df_mg['TP_SEXO'] == 'F']

media = pd.to_numeric(df_montes_claros_fem['NU_NOTA_MT']).mean()
print(f'Media = {media}')


Media = 562.988836104513


### 6) Qual é a média da nota em Matemática dos alunos que estudaram numa escola no município de Sabará e que possuem TV por assinatura na residência?

- Codigo de Sabara: 3156700
- NU_NOTA_MT: Nota da prova de Matemática
- Q021: Na sua residência tem TV por assinatura?
- - A: Nao
- - B: Sim

In [9]:
df_sabara = df_mg.loc[df_mg['CO_MUNICIPIO_ESC'] == 3156700]
df_sabara_com_tv =  df_sabara.loc[df_sabara['Q021'] == 'B']

media = pd.to_numeric(df_sabara_com_tv['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 580.102985074627


### 7) Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola em Minas Gerais e que possuem dois fornos micro-ondas em casa?

- Q016: Na sua residência tem forno micro-ondas?
  - A: Nao
  - B: Sim, um
  - C: Sim, dois
  - D: Sim, tres
  - E: Sim, quatro ou mais

In [14]:
df_mg_micro_ondas = df_mg.loc[df_mg['Q016'] == 'C']

media = pd.to_numeric(df_mg_micro_ondas['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 601.4825757575757


### 8) Qual é a nota média em Matemática dos alunos que estudaram numa escola em Minas Gerais e cuja mãe completou a pós-graduação?

- NU_NOTA_MT:
- Q002: Até que série sua mãe, ou a mulher responsável por você, estudou?
  - G: Completou a Pós-graduação.


In [18]:
df_mg_mae = df_mg.loc[df_mg['Q002'] == 'G']

media = pd.to_numeric(df_mg_mae['NU_NOTA_MT']).mean()
print(f'Media = {media}')


Media = 642.2451844866204


### 9) Qual é a nota média em Matemática dos alunos que estudaram numa escola em Belo Horizonte ou em Conselheiro Lafaiete?

- Belo Horizonte: 3106200
- Conselheiro Lafaiete: 3118304

In [21]:
df_municipios = df_mg.loc[df_mg['CO_MUNICIPIO_ESC'].isin([3106200, 3118304])]
df_municipios

media = pd.to_numeric(df_municipios['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 621.0282524049474


### 10) Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola em Minas Gerais e que moram sozinhos?

- Q005: Incluindo você, quantas pessoas moram atualmente em sua residência?
  - 1: 1, pois moro sozinho(a).

In [25]:
df_sozinho = df_mg.loc[df_mg['Q005'] == 1]
df_sozinho

media = pd.to_numeric(df_sozinho['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 537.0544303797469


### 11) Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola em Minas Gerais, cujo pai completou pós-graduação e possuem renda familiar entre R$ 8.360,01 e R$ 9.405,00?

- Q006: Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.)
  - L: De R$ 8.360,01 até R$ 9.405,00
- Q001: Até que série seu pai, ou o homem responsável por você, estudou?  
  - G: Completou a Pós-graduação.

In [31]:
df_renda = df_mg.loc[df_mg['Q006'] == 'L']
df_renda_grad = df_renda.loc[df_renda['Q001'] == 'G']


media = pd.to_numeric(df_renda_grad['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 623.9422535211268


### 12) Qual é a nota média em Matemática dos alunos do sexo Feminino que estudaram numa escola em Lavras e escolheram “Espanhol” como língua estrangeira?

- Lavras: 3138203
- TP_LINGUA: Lingua Estrangeira

In [35]:
df_fem_mg = df_mg.loc[df_mg['TP_SEXO'] == 'F']
df_fem_lavras = df_fem_mg.loc[df_fem_mg['CO_MUNICIPIO_ESC'] == 3138203]
df_fem_lavras_esp = df_fem_lavras.loc[df_fem_lavras['TP_LINGUA'] == 1]


media = pd.to_numeric(df_fem_lavras_esp['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 513.3833333333333


### 13) Qual é a nota média em Matemática dos alunos do sexo Masculino que estudaram numa escola em Ouro Preto?

- Ouro Preto: 3146107

In [36]:
df_masc_mg = df_mg.loc[df_mg['TP_SEXO'] == 'M']
df_masc_ouro_preto = df_masc_mg.loc[df_masc_mg['CO_MUNICIPIO_ESC'] == 3146107]

media = pd.to_numeric(df_masc_ouro_preto['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 615.5823529411764


### 14) Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola em Minas Gerais e possuem entre 26 e 30 anos?

- TP_FAIXA_ETARIA: Faixa Etaria
  - 11: Entre 26 e 30 anos

In [4]:
df_etario = df_mg.loc[df_mg['TP_FAIXA_ETARIA'] == 11]

media = pd.to_numeric(df_etario['NU_NOTA_CH']).mean()
print(f'Media = {media}')

Media = 473.477922077922


### 15) Qual é a nota média em Matemática dos alunos do sexo FEMININO, que estudaram numa escola em Belo Horizonte, Sabará, Nova Lima ou Betim e estudaram em uma escola privada?

- Belo Horizonte: 3106200
- Sabara: 3156700
- Nova Lima: 3144805
- Betim: 3106705

- TP_ESCOLA: Tipo de escola do Ensino Médio
  - 3: Privada

In [10]:
df_municipios = df_mg.loc[df_mg['CO_MUNICIPIO_ESC'].isin([3106200, 3156700, 3144805, 3106705])]
df_municipios_privado = df_municipios.loc[df_municipios['TP_ESCOLA'] == 3]
df_municipios_privado_fem = df_municipios_privado.loc[df_municipios_privado['TP_SEXO'] == 'F']


media = pd.to_numeric(df_municipios_privado_fem['NU_NOTA_MT']).mean()
print(f'Media = {media}')

Media = 679.1369481765835
